In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import requests

#### 1. scraping the wiki page

In [2]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', verify=False).text

data = BeautifulSoup(wiki, 'lxml')

table = data.find_all("table")
table1 = table[0]
tr  = table1.find_all("tr")
th = tr[0].find_all("th")


C:\Users\dsamet\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


#### 2. Getting the columns

In [3]:


columns = []


for cell in th:
    header = cell.text
    header = header.strip()
    columns.append(header)
columns

['Postcode', 'Borough', 'Neighbourhood']

#### 3. getting all the rows

In [4]:

rowdata = []
for row in tr[2:]:
    dt  = ()
    cells = row.find_all("td")
    for cell in cells:
        font = cell.find("font")
        if font is not None:
            celldata = font.text
        else:
            celldata = cell.text
        celldata = celldata.strip()
        celldata = celldata.replace("\n"," ")
        dt = dt+(celldata,)
    rowdata.append(dt)

#### 4. Creating a DataFrame out of it all

In [5]:
df = pd.DataFrame(rowdata, columns = columns)

#### 5. Removing all where Borough is Not Assigned

In [6]:
df.drop( df[df['Borough'] == 'Not assigned'].index, inplace=True)

#### 6. Updating all where Neighbourhood is Not Assigned to be equell to the Borough

In [7]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

#### 7. Grouping by Postcode and Borough

In [8]:
df1 = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df1.shape

(103, 3)